In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta
from biogeme.expressions import log
import math

In [2]:
# read the data
df = pd.read_csv("data/ky_estdata.csv")
df.head()

,Unnamed: 0,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,...,EDU_ORIG,MED_ORIG,ENT_ORIG,JOBS_IND_HOSFOOD_ORIG,MISC_ORIG,ADM.1_ORIG,JOBS_EDU_NOHS_ORIG,JOBS_EDU_HS_ORIG,JOBS_EDU_NOBACH_ORIG,JOBS_EDU_BACH_ORIG
0,0,P,2018GQ0000289,6,1,800,3,21,1013097,69,...,3259,5613,161,3103,578,1686,3315,10113,8855,5408
1,1,P,2018GQ0000475,6,1,2200,3,21,1013097,121,...,34755,68360,7481,44186,12992,11022,40679,108224,122145,102563
2,2,P,2018GQ0000520,6,1,500,3,21,1013097,62,...,6469,9610,822,7312,1470,1997,5416,14733,14982,11331
3,3,P,2018GQ0000533,6,1,1901,3,21,1013097,88,...,5030,5937,758,4643,1300,2074,4903,15067,12674,7907
4,4,P,2018GQ0000676,6,1,500,3,21,1013097,73,...,6469,9610,822,7312,1470,1997,5416,14733,14982,11331


In [3]:
puma_acs_data = pd.read_csv("data/ACS_2018.csv")
puma_lodes_data = pd.read_csv("data/wac_puma.csv")

puma_acs_data["Geo_FIPS"] = puma_acs_data["Geo_FIPS"].astype(str).str.zfill(7)
puma_acs_data = puma_acs_data.set_index("Geo_FIPS")

puma_lodes_data["puma"] = puma_lodes_data["puma"].astype(str).str.zfill(7)
puma_lodes_data = puma_lodes_data.set_index("puma")

In [4]:
distances = pd.read_csv("data/ky_distances.csv").set_index("Unnamed: 0")
distances.columns

Index(['2100100', '2100200', '2101500', '2100300', '2101400', '2100600',
       '2101100', '2100700', '2102000', '2101200', '2100900', '2101000',
       '2102800', '2101800', '2102300', '2102700', '2100800', '2102600',
       '2101600', '2102100', '2102500', '2102200', '2101901', '2101902',
       '2102400', '2101704', '2101705', '2101703', '2101702', '2101701',
       '2101706', '2100400', '2101300', '2100500'],
      dtype='object')

In [5]:
# clean up the database
df = df.select_dtypes(['number'])
df = df.fillna(0)

In [6]:
df['CHOSEN_PUMA'] = df['CHOSEN']
df['CHOSEN'] = 0
for i in range(1, 35): 
    var = 'ALT' + str(i) + '_PUMA'
    df['CHOSEN'] = np.where(df[var]==df['CHOSEN_PUMA'], i, df['CHOSEN'])

In [7]:
df["IN_COLLEGE"] = np.where(df["SCHG"] == 15, 1, 0)
df["IN_COLLEGE"]

0       0
1       1
2       1
3       1
4       1
       ..
3988    0
3989    0
3990    0
3991    0
3992    0
Name: IN_COLLEGE, Length: 3993, dtype: int32

In [8]:
df["AGE_18_34"] = np.where(df["AGEP"] <= 34, 1, 0)
df["AGE_35_64"] = np.where((df["AGEP"] >= 35) & (df["AGEP"] <= 64), 1, 0)
df["AGE_OVER_65"] = np.where((df["AGEP"] >= 65), 1, 0)
df["FOREIGN"] = np.where(df["NATIVITY"] == 2, 1, 0)

In [9]:
database = db.Database('ky_data', df)

In [10]:
# The following statement allows you to use the names of the
# variable as Python variable.
globals().update(database.variables)

In [11]:
puma_acs_data["UNEMP"] = puma_acs_data["Population 16 Years and Over in Labor Force Civilian Unemployed"] / puma_acs_data["Population 16 Years and Over in Labor Force Civilian"]
puma_acs_data["COLLEGE"] = puma_acs_data["Population 3 Years and Over Enrolled in School Private School College"] + puma_acs_data["Population 3 Years and Over Enrolled in School Public School College"]
puma_acs_data["VACANCY_PCT"] = puma_acs_data["Housing Units Vacant"] / puma_acs_data["Housing Units"]

In [12]:
# Parameters to be estimated
c_emp=Beta('c_emp', 0, None, None, 0)
c_dist=Beta('c_dist', 0, None, None, 0)
c_logdist=Beta('c_logdist', 0, None, None, 0)
c_time=Beta('c_time', 0, None, None, 0)
c_unemp = Beta("c_unemp", 0, None, None, 0)
c_hhinc = Beta("c_hhinc", 0, None, None, 0)
c_internal = Beta("c_internal", 0, None, None, 0)
c_urban = Beta("c_urban", 0, None, None, 0)
c_huprice = Beta("c_huprice", 0, None, None, 0)
c_hurent = Beta("c_hurent", 0, None, None, 0)
c_vacancy = Beta("c_vacancy", 0, None, None, 0)
c_college = Beta("c_college", 0, None, None, 0)
c_age_18_34 = Beta("c_age_18_34", 0, None, None, 0)
c_age_35_64 = Beta("c_age_35_64", 0, None, None, 0)
c_age_OVER_65 = Beta("c_age_OVER_65", 0, None, None, 0)
c_foreign = Beta("c_foreign", 0, None, None, 0)


# V0 = c_inc * PINC + c_age1830 * AGE1830 + c_married * MARRIED

In [13]:
df.columns

Index(['Unnamed: 0', 'DIVISION', 'SPORDER', 'PUMA', 'REGION', 'ST', 'ADJINC',
       'PWGTP', 'AGEP', 'CIT',
       ...
       'JOBS_EDU_NOHS_ORIG', 'JOBS_EDU_HS_ORIG', 'JOBS_EDU_NOBACH_ORIG',
       'JOBS_EDU_BACH_ORIG', 'CHOSEN_PUMA', 'IN_COLLEGE', 'AGE_18_34',
       'AGE_35_64', 'AGE_OVER_65', 'FOREIGN'],
      dtype='object', length=490)

In [23]:
for i, puma in enumerate(distances.columns):
    num = i + 1
    initialization = "V{0} = log(int(puma_acs_data.loc['{1}', 'Total Population'])) + c_dist * ALT{0}_DIST + c_logdist * log(ALT{0}_DIST + 1) + c_hhinc * int(puma_acs_data.loc['{1}', 'Median Household Income (In 2019 Inflation Adjusted Dollars)']) + c_urban / DENS_ORIG * (puma_acs_data.loc['{1}', 'Population Density (Per Sq. Mile)'] - DENS_ORIG) + c_hurent * int(puma_acs_data.loc['{1}', 'Median Gross Rent']) + c_college * IN_COLLEGE * int(puma_acs_data.loc['{1}', 'COLLEGE']) + c_vacancy * float(puma_acs_data.loc['{1}', 'VACANCY_PCT']) + c_foreign * FOREIGN * int(puma_acs_data.loc['{1}', 'Total Population Foreign Born']) + c_age_18_34 * AGE_18_34 * int(puma_acs_data.loc['{1}', 'Total Population 18 to 34 Years']) / int(puma_acs_data.loc['{1}', 'Total Population']) + c_age_35_64 *AGE_35_64 * int(puma_acs_data.loc['{1}', 'Total Population 35 to 64 Years']) / int(puma_acs_data.loc['{1}', 'Total Population']) + c_age_OVER_65 * AGE_OVER_65 * int(puma_acs_data.loc['{1}', 'Total Population 65 and Over']) / int(puma_acs_data.loc['{1}', 'Total Population'])".format(num, puma)
    exec(initialization)
print(V1)

(((((((((((log(`196280`) + (c_dist(-9.54774355809149e-06) * ALT1_DIST)) + (c_logdist(0) * log((ALT1_DIST + `1`)))) + (c_hhinc(1.694784870156998e-05) * `45561`)) + ((c_urban(-0.01590436806676023) / DENS_ORIG) * (`236.724896654487` - DENS_ORIG))) + (c_hurent(-0.002206947637847175) * `700`)) + ((c_college(0.00016196643456364604) * IN_COLLEGE) * `12344`)) + (c_vacancy(1.409723109120851) * `0.1783103294515363`)) + ((c_foreign(3.634816748699458e-05) * FOREIGN) * `2733`)) + (((c_age_18_34(0) * AGE_18_34) * `41282`) / `196280`)) + (((c_age_35_64(0) * AGE_35_64) * `74145`) / `196280`)) + (((c_age_OVER_65(0) * AGE_OVER_65) * `39562`) / `196280`))


In [24]:
# Associate utility functions with the numbering of alternatives
V = {1: V1, 2: V2, 3: V3, 4: V4, 5: V5, 6: V6, 7: V7, 8: V8, 9: V9, 10: V10, 11: V11, 12: V12, 13: V13, 14: V14, 15: V15, 16: V16, 17: V17, 18: V18, 19: V19, 20: V20, 21: V21, 22: V22, 23: V23, 24: V24, 25: V25, 26: V26, 27: V27, 28: V28, 29: V29, 30: V30, 31: V31, 32: V32, 33: V33, 34: V34}

# Associate the availability conditions with the alternatives
#av = {1: AV1, 2: AV2, 3: AV3, 4: AV4, 5: AV5, 6: AV6, 7: AV7, 8: AV8, 9: AV9, 10: AV10, 11: AV11, 12: AV12, 13: AV13, 14: AV14, 15: AV15, 16: AV16, 17: AV17, 18: AV18, 19: AV19, 20: AV20, 21: AV21, 22: AV22, 23: AV23, 24: AV24, 25: AV25, 26: AV26, 27: AV27, 28: AV28, 29: AV29, 30: AV30, 31: AV31, 32: AV32, 33: AV33, 34: AV34}
av = {}
for i in range(1, 35):
    av[i] = 1

# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
logprob = models.loglogit(V, av, CHOSEN)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'ky_final1'

# Calculate the null log likelihood for reporting.
biogeme.calculateNullLoglikelihood(av)

# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

                  Value       Std err     t-test       p-value  Rob. Std err  \
c_age_18_34    3.943581  7.321138e-01   5.386568  7.181580e-08  7.382175e-01   
c_age_35_64   -4.154439  1.589953e+00  -2.612932  8.976922e-03  1.567709e+00   
c_age_OVER_65 -1.258992  2.831537e+00  -0.444632  6.565856e-01  2.844319e+00   
c_college      0.000138  9.501173e-06  14.527768  0.000000e+00  9.149896e-06   
c_dist        -0.000008  4.087109e-07 -19.134180  0.000000e+00  5.351840e-07   
c_foreign      0.000040  1.669614e-05   2.366969  1.793445e-02  1.385364e-05   
c_hhinc        0.000024  3.686906e-06   6.621078  3.565903e-11  3.681668e-06   
c_hurent      -0.002679  3.472709e-04  -7.713542  1.221245e-14  3.553084e-04   
c_logdist     -0.212328  6.291619e-03 -33.747724  0.000000e+00  7.586219e-03   
c_urban       -0.020169  5.865252e-03  -3.438715  5.844832e-04  6.426160e-03   
c_vacancy      1.801158  6.590677e-01   2.732887  6.278184e-03  6.268338e-01   

               Rob. t-test  Rob. p-valu

In [16]:
# start from there and add -- for destination choice
# UNEMPLOYMENT RATE
# MED RENT
# MED HOUSE PRICE
# RURAL to RURAL
# RURAL to URBAN
# URBAN to URBAN 
# URBAN to RURAL
# MEDIAN INCOME of alternative PUMA
# VACANCY RATE (VACANCY / HOUSING UNITS)
# RENT as PERCENT OF INCOME in DEST -- better to do median rent as a percent of my income
# OWNERSHIP COST as PERCENT OF INCOME -- median home price as a percent of my income



# % of POP AGE 18-30 - if I"m 18-30, am I more likely to go there
# % of POP AGE 65+ - if I"m age 65+
# INCOME(of the person) * URBAN (of the alternative)

In [17]:
# then add one more alternative for: Didn't move.  